In [ ]:
# default_exp core

# module name here

> API details.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import math
import logging

import torch
import torch.nn as nn
import torch.utils.model_zoo as model_zoo

In [ ]:
BN_MOMENTUM = 0.1
logger = logging.getLogger(__name__)

model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}


In [ ]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)


In [ ]:
conv3x3(2,5)

Conv2d(2, 5, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [ ]:

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes, momentum=BN_MOMENTUM)
        self.downsample = downsample # this is a layer ...
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        
        # so if number of input channels doesnt equal number of output channels, the downsample 
        # function passed in need to make it right so you can add the residual to the output!
        if self.downsample is not None:
            residual = self.downsample(x)
        
        print("residual size : {}".format(residual.size()))
        print("out size : {}".format(out.size()))

        out += residual
        out = self.relu(out)

        return out
    
class PkuCenterNet(nn.Module):
    def __init__(self):
        self.inplanes = 64
    # block - BlockType
    # blocks - how many times to repeat BasicBlock or bottleneck
    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion, momentum=BN_MOMENTUM),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        print("Inplanes now set to : {}".format(self.inplanes))
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

In [ ]:
# input channels // output channels
# these kinda have to match to make the output and residual be able to sum together ...
mdl = BasicBlock(3,3,stride=1)

# layers ...
cnet = PkuCenterNet()
layer1 = cnet._make_layer(BasicBlock, 64, 2)
layer2 = cnet._make_layer(BasicBlock, 128, 2)
layer3 = cnet._make_layer(BasicBlock, 256, 2)


Inplanes now set to : 64
Inplanes now set to : 128
Inplanes now set to : 256


In [ ]:
# x is NCHW
x1 = torch.rand(4,3,28,28)
x2 = torch.rand(4,64,28,28)

In [ ]:
mdl(x1).size()

residual size : torch.Size([4, 3, 28, 28])
out size : torch.Size([4, 3, 28, 28])


torch.Size([4, 3, 28, 28])

In [ ]:
print(x2.size())
x2 = layer1(x2)
print(x2.size())
x2 = layer2(x2)
print(x2.size())
x2 = layer3(x2)


torch.Size([4, 64, 28, 28])
residual size : torch.Size([4, 64, 28, 28])
out size : torch.Size([4, 64, 28, 28])
residual size : torch.Size([4, 64, 28, 28])
out size : torch.Size([4, 64, 28, 28])
torch.Size([4, 64, 28, 28])
residual size : torch.Size([4, 128, 28, 28])
out size : torch.Size([4, 128, 28, 28])
residual size : torch.Size([4, 128, 28, 28])
out size : torch.Size([4, 128, 28, 28])
torch.Size([4, 128, 28, 28])
residual size : torch.Size([4, 256, 28, 28])
out size : torch.Size([4, 256, 28, 28])
residual size : torch.Size([4, 256, 28, 28])
out size : torch.Size([4, 256, 28, 28])


In [ ]:
layer3

Sequential(
  (0): BasicBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (downsample): Sequential(
      (0): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (1): BasicBlock(
    (conv1): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn2): BatchNorm2d(256, eps=1

In [ ]:
# Lets do transpose now ..
planes = 2
up = nn.ConvTranspose2d(
                    in_channels=planes,
                    out_channels=planes,
                    kernel_size=kernel,
                    stride=2,
                    padding=padding,
                    output_padding=output_padding,
                    bias=self.deconv_with_bias)

In [ ]:
class nuttin():
    def __init__(self):
        self.x = 64
    

In [ ]:
n = nuttin()


In [ ]:
n.__setattr__('a','a')

In [ ]:
n.a

'a'